In [16]:
import mesh_utils
import numpy as np
import importlib
importlib.reload(mesh_utils)

<module 'mesh_utils' from '/Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/tools/mesh_utils.py'>

## For converting the raw .vtp and .vtu files to openCARP format:

In [ ]:
data_path = "../data/instance_001/"
vtp_path = data_path + "instance_001.vtp"
vtu_path = data_path + "instance_001.vtu"
output_prefix = "instance_001"

mesh_utils.vtk_to_opencarp(vtp_path, vtu_path, output_prefix, data_path)

Reading surface mesh: ../data/instance_001/instance_001.vtp
Reading volume mesh: ../data/instance_001/instance_001.vtu
Converting to openCARP format. Output directory: ../data/instance_001/
Extracted tv shape: (478820,)
tv range: 0.0 to 1.0
Extracted tm shape: (478820,)
tm range: 0.0 to 1.0
Extracted rtSin shape: (478820,)
rtSin range: -1.0 to 1.0
Extracted rtCos shape: (478820,)
rtCos range: -1.0 to 0.9999998807907104
Extracted rt shape: (478820,)
rt range: 1.2781016494045616e-06 to 0.999995768070221
Extracted ab shape: (478820,)
ab range: 0.0 to 1.0
Saved UVC data to ../data/instance_001/instance_001_UVC.csv
Created ../data/instance_001/instance_001.pts with 478820 points
Created instance_001.elem with tetrahedral elements
Created instance_001.surf with surface triangles
Conversion complete! Files saved to: ../data/instance_001/


## Read in the points, surfaces, tetrahedra, and UVCs

In [17]:
data_path = "/Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/"
output_prefix = "instance_001_lowres"
reader = mesh_utils.OpenCARPMeshReader(data_path, output_prefix)
points, tetrahedra, tetrahedra_regions, triangles, triangle_regions, uvc_data = reader.read_all()

Reading points from /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/instance_001_lowres.pts
Loaded 191568 points with shape (191568, 3)
Reading tetrahedra from /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/instance_001_lowres.elem
Loaded 974607 tetrahedra with shape (974607, 4)
Reading triangles from /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/instance_001_lowres.surf
Loaded 127944 triangles with shape (127944, 3)
Reading UVC data from /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/instance_001_lowres_UVC.csv
Loaded UVC data with shape (191568, 6)
UVC columns: ['tv', 'tm', 'rtSin', 'rtCos', 'rt', 'ab']


## Use the UVCs to define a coordinate system and compute the fiber and sheet directions

In [18]:
phi_transmural = np.array(uvc_data['tm'])
phi_longitudinal = np.array(uvc_data['ab'])
phi_circumferential = np.array(uvc_data["rt"])
TransmuralField, LongitudinalField, CircumferentialField = mesh_utils.compute_normed_gradients(points, -phi_transmural, -phi_longitudinal, phi_circumferential)

In [24]:
mesh_utils.visualize_vector_fields(
    points, triangles, triangle_regions,
    TransmuralField, LongitudinalField, CircumferentialField, subsample_factor=100, glyph_scale=5000
)

Original points: 191568
Original triangles: 127944
Epicardium triangles: 61801
Endocardium triangles: 63850
Sampled epicardium points: 310
Sampled endocardium points: 321


Widget(value='<iframe src="http://localhost:61460/index.html?ui=P_0x34b7a8bb0_11&reconnect=auto" class="pyvist…

In [25]:
fiber_dirs, sheet_dirs, sheet_normal_dirs = mesh_utils.compute_fiber_sheet_directions(TransmuralField, LongitudinalField, CircumferentialField, phi_transmural, 
                                                                                      endo_fiber_angle=60.0, epi_fiber_angle=-60, endo_sheet_angle=-65, epi_sheet_angle=25
)

mesh_utils.visualize_fibers(
    points, triangles, triangle_regions,
    fiber_dirs, subsample_factor=50, glyph_scale=5000
)

Original points: 191568
Original triangles: 127944
Epicardium triangles: 61801
Endocardium triangles: 63850
Sampled epicardium points: 621
Sampled endocardium points: 642


Widget(value='<iframe src="http://localhost:61460/index.html?ui=P_0x3343584c0_12&reconnect=auto" class="pyvist…

In [26]:
mesh_utils.write_lon_file(
    f"{data_path}{output_prefix}.lon", 
    fiber_dirs, 
    sheet_dirs, 
    sheet_normal_dirs,  # Now including sheet normal directions
    tetrahedra
)

Processing 974607 elements for .lon file...


Precomputing element directions: 100%|██████████| 974607/974607 [00:15<00:00, 62264.72it/s]


Verifying orthogonality of element directions...
Maximum dot products: fiber·sheet=0.999750, fiber·normal=0.999961, sheet·normal=0.999033
Writing to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/instance_001_lowres.lon...


Writing .lon file: 100%|██████████| 98/98 [00:02<00:00, 47.64it/s]

Successfully wrote fiber orientations to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/instance_001_lowres.lon


## Tag the fast conducting endocardium

In [27]:
mesh_utils.tag_fast_conducting_endocardium(
    points, 
    tetrahedra, 
    triangles, 
    triangle_regions, 
    phi_longitudinal, 
    f"{data_path}{output_prefix}.elem",
    long_min = 0.2,
    long_max = 0.9
)

Identifying points with phi_longitudinal between 0.2 and 0.9...
Found 154765 points with valid longitudinal coordinate
Identifying endocardial points...
Found 32143 points on the endocardium
Found 25493 points that satisfy both criteria
Tagging tetrahedra...


100%|██████████| 98/98 [00:00<00:00, 142.00it/s]


Tagged 163358 tetrahedra (16.76%) as fast conducting endocardium
Writing modified element file to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/instance_001_lowres.elem...


100%|██████████| 974607/974607 [00:01<00:00, 895907.04it/s]


Successfully wrote modified element file to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/instance_001_lowres.elem


## Tag the fascicular sites and output to a .vtx file

In [29]:
import numpy as np
from scipy.cluster.vq import kmeans2

def filter_fascicular_sites_closer_to_LV(points, is_fascicular_site, septal_normal, LV_center, RV_center):
    """
    Filter fascicular sites to keep only those closer to the left ventricle.
    """
    # Get indices and coordinates of all fascicular sites
    fascicular_indices = np.where(is_fascicular_site)[0]
    fascicular_points = points[fascicular_indices]
    
    print(f"Total fascicular sites: {len(fascicular_indices)}")
    
    if len(fascicular_indices) == 0:
        print("No fascicular sites found.")
        return is_fascicular_site
    
    # If there's only one cluster, check if it's closer to LV or RV
    if len(fascicular_indices) < 5:  # Not enough points for reliable clustering
        # Compute signed distance to the plane separating LV and RV
        # The plane passes through the middle point between LV and RV centers
        # with septal_normal as its normal
        mid_point = (LV_center + RV_center) / 2
        
        # For each fascicular point, calculate its position relative to the plane
        # Positive values mean the point is on the LV side
        distances = np.dot(fascicular_points - mid_point, septal_normal)
        
        if np.all(distances > 0) or np.all(distances < 0):
            # All points are on the same side
            is_lv_side = np.mean(distances) > 0
            print(f"All fascicular sites are on the {'LV' if is_lv_side else 'RV'} side.")
            if not is_lv_side:
                # If all points are on RV side, return empty array
                return np.zeros_like(is_fascicular_site, dtype=bool)
            return is_fascicular_site
        
    # Use k-means clustering to separate the two disks
    centroids, labels = kmeans2(fascicular_points, k=2, minit='points')
    
    # Determine which centroid is closer to LV
    # Calculate vectors from RV center to each centroid
    vectors_to_centroids = centroids - RV_center
    
    # Project these vectors onto the septal normal
    projections = np.dot(vectors_to_centroids, septal_normal)
    
    # The centroid with larger projection is closer to LV
    lv_cluster_label = np.argmax(projections)
    
    # Count points in each cluster
    cluster0_count = np.sum(labels == 0)
    cluster1_count = np.sum(labels == 1)
    print(f"Cluster 0: {cluster0_count} points, Cluster 1: {cluster1_count} points")
    print(f"Selected LV cluster (label {lv_cluster_label}) with {np.sum(labels == lv_cluster_label)} points")
    
    # Create a new boolean array with only the LV cluster points
    is_lv_fascicular_site = np.zeros_like(is_fascicular_site, dtype=bool)
    lv_indices = fascicular_indices[labels == lv_cluster_label]
    is_lv_fascicular_site[lv_indices] = True
    
    return is_lv_fascicular_site

disk_height_tolerance = 0.05
disk_radius = 0.02

coord_t = 0.65
coord_l = 0.30
coord_c = 0.8

is_fascicular_site = np.logical_and( np.abs(phi_transmural - coord_t) < disk_height_tolerance , ((phi_longitudinal - coord_l)**2.0 + (phi_circumferential - coord_c)**2.0) < disk_radius**2.0 ) 

LV_points = np.array(list(set(triangles[triangle_regions == 3].flatten()))) # Left ventricular endocardium
RV_points = np.array(list(set(triangles[triangle_regions == 4].flatten()))) # Right ventricular endocardium

LV_center = points[LV_points].mean(axis=0)
RV_center = points[RV_points].mean(axis=0)

septal_normal = RV_center - LV_center
septal_normal /= np.linalg.norm(septal_normal)

is_fascicular_site = filter_fascicular_sites_closer_to_LV(points, is_fascicular_site, septal_normal, LV_center, RV_center)

mesh_utils.visualize_fascicular_sites(
    points, triangles, triangle_regions,
    is_fascicular_site, sphere_scale=1000
)

mesh_utils.save_fascicular_sites_to_vtx(is_fascicular_site, output_filename=f"{data_path}fascicular_stim.vtx")

Total fascicular sites: 11
Cluster 0: 6 points, Cluster 1: 5 points
Selected LV cluster (label 1) with 5 points
Original points: 191568
Original triangles: 127944
Number of fascicular sites: 5
Endocardium triangles: 63850
Displaying all 5 fascicular sites


/Users/jamesmcgreivy/miniconda3/envs/openCARP/lib/python3.10/site-packages/pyvista/core/filters/data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:61460/index.html?ui=P_0x338d55e70_14&reconnect=auto" class="pyvist…

Saving 5 fascicular site indices to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/fascicular_stim.vtx
Successfully saved fascicular sites to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/fascicular_stim.vtx


'/Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/fascicular_stim.vtx'

##  Extra utility function -- for visualizing the scalar fields

In [15]:
mesh_utils.visualize_phi(points, phi_longitudinal, subsample_factor=1, point_size=5)

Original points: 478820
Subsampling points from 478820 to 50000


Widget(value='<iframe src="http://localhost:61460/index.html?ui=P_0x17832efe0_5&reconnect=auto" class="pyvista…